In [1]:
DATA_PATH = '/Users/vahid/data/recommender/'

In [2]:
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [4]:
df = spark.read.format("csv").option("header", "true").load(DATA_PATH + 'train_interactions.csv')
df = df.sample(fraction=0.005, seed=0)
df.count()

491592

In [5]:
df = df.drop('interaction_min')
df = df.drop('delivery_min')

In [6]:
coloumsn = ["interaction_dow", "interaction_hour", "delivery_dow", "delivery_hour"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in  coloumsn]

pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

In [7]:
print(df.count())
df.show(3)

491592
+-------+--------+-----------+---------------+----------------+------------+-------------+---------------------+----------------------+------------------+-------------------+
|user_id|notif_id|interaction|interaction_dow|interaction_hour|delivery_dow|delivery_hour|interaction_dow_index|interaction_hour_index|delivery_dow_index|delivery_hour_index|
+-------+--------+-----------+---------------+----------------+------------+-------------+---------------------+----------------------+------------------+-------------------+
|8855709|  473050|          0|              2|               0|           1|           23|                  0.0|                  23.0|               0.0|               21.0|
|4562680|  473073|          0|              2|               7|           1|           23|                  0.0|                  11.0|               0.0|               21.0|
|3409532|  519584|          0|              1|               1|           1|            0|                  1.0|      

In [12]:
df = df.drop('interaction_dow').drop('interaction_hour').drop('delivery_dow').drop('delivery_hour')

In [8]:
encoder = OneHotEncoderEstimator(inputCols=["interaction_dow_index", "interaction_hour_index", "delivery_dow_index", "delivery_hour_index"],
                                 outputCols=["interaction_dow_vec", "interaction_hour_vec", "delivery_dow_vec", "delivery_hour_vec"])
model = encoder.fit(df)
df = model.transform(df)

In [9]:
df.show(3)

+-------+--------+-----------+---------------+----------------+------------+-------------+---------------------+----------------------+------------------+-------------------+-------------------+--------------------+----------------+-----------------+
|user_id|notif_id|interaction|interaction_dow|interaction_hour|delivery_dow|delivery_hour|interaction_dow_index|interaction_hour_index|delivery_dow_index|delivery_hour_index|interaction_dow_vec|interaction_hour_vec|delivery_dow_vec|delivery_hour_vec|
+-------+--------+-----------+---------------+----------------+------------+-------------+---------------------+----------------------+------------------+-------------------+-------------------+--------------------+----------------+-----------------+
|8855709|  473050|          0|              2|               0|           1|           23|                  0.0|                  23.0|               0.0|               21.0|      (6,[0],[1.0])|          (23,[],[])|   (6,[0],[1.0])|  (23,[21],[1.0

In [10]:
df = df.drop('interaction_dow_index').drop('interaction_hour_index').drop('delivery_dow_index').drop('delivery_hour_index')

In [13]:
df.show(3)

+-------+--------+-----------+-------------------+--------------------+----------------+-----------------+
|user_id|notif_id|interaction|interaction_dow_vec|interaction_hour_vec|delivery_dow_vec|delivery_hour_vec|
+-------+--------+-----------+-------------------+--------------------+----------------+-----------------+
|8855709|  473050|          0|      (6,[0],[1.0])|          (23,[],[])|   (6,[0],[1.0])|  (23,[21],[1.0])|
|4562680|  473073|          0|      (6,[0],[1.0])|     (23,[11],[1.0])|   (6,[0],[1.0])|  (23,[21],[1.0])|
|3409532|  519584|          0|      (6,[1],[1.0])|     (23,[22],[1.0])|   (6,[0],[1.0])|       (23,[],[])|
+-------+--------+-----------+-------------------+--------------------+----------------+-----------------+
only showing top 3 rows



In [14]:
data = df.drop('user_id').drop('notif_id')

In [15]:
data.show(3)

+-----------+-------------------+--------------------+----------------+-----------------+
|interaction|interaction_dow_vec|interaction_hour_vec|delivery_dow_vec|delivery_hour_vec|
+-----------+-------------------+--------------------+----------------+-----------------+
|          0|      (6,[0],[1.0])|          (23,[],[])|   (6,[0],[1.0])|  (23,[21],[1.0])|
|          0|      (6,[0],[1.0])|     (23,[11],[1.0])|   (6,[0],[1.0])|  (23,[21],[1.0])|
|          0|      (6,[1],[1.0])|     (23,[22],[1.0])|   (6,[0],[1.0])|       (23,[],[])|
+-----------+-------------------+--------------------+----------------+-----------------+
only showing top 3 rows

